In [1]:
import sys
import os

In [2]:
# sys.path.append('../..')
sys.path.insert(0, os.path.abspath('..'))


In [3]:
sys.path

['/home/zhanwen/gsc',
 '/home/zhanwen/gsc/notebooks',
 '/home/zhanwen/miniconda3/envs/sgb/lib/python38.zip',
 '/home/zhanwen/miniconda3/envs/sgb/lib/python3.8',
 '/home/zhanwen/miniconda3/envs/sgb/lib/python3.8/lib-dynload',
 '',
 '/home/zhanwen/miniconda3/envs/sgb/lib/python3.8/site-packages',
 '/home/zhanwen/miniconda3/envs/sgb/lib/python3.8/site-packages/pycocotools-2.0-py3.8-linux-x86_64.egg',
 '/home/zhanwen/miniconda3/envs/sgb/lib/python3.8/site-packages/fonttools-4.33.3-py3.8.egg',
 '/home/zhanwen/gsc',
 '/home/zhanwen/miniconda3/envs/sgb/lib/python3.8/site-packages/Pillow-9.0.1-py3.8-linux-x86_64.egg',
 '/home/zhanwen/miniconda3/envs/sgb/lib/python3.8/site-packages/torchvision-0.12.0a0-py3.8-linux-x86_64.egg']

In [4]:
os.listdir()

['Examine Model Componenets.ipynb', 'Untitled.ipynb', '.ipynb_checkpoints']

In [5]:
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.config import cfg


In [11]:
def fix_eval_modules(eval_modules):
    for module in eval_modules:
        for _, param in module.named_parameters():
            param.requires_grad = False

In [6]:
config_vgg = '/home/zhanwen/gsc/checkpoints/gbnet_vgg_1/config.yml'

In [7]:
cfg.merge_from_file(config_vgg)
cfg.OUTPUT_DIR = '/home/zhanwen/gsc/checkpoints/gbnet_vgg_1'
cfg.freeze()

In [8]:
from pprint import pp

In [9]:
pp(cfg)

{'MODEL': {'FLIP_AUG': False,
           'RPN_ONLY': False,
           'MASK_ON': False,
           'RETINANET_ON': False,
           'KEYPOINT_ON': False,
           'ATTRIBUTE_ON': False,
           'RELATION_ON': True,
           'DEVICE': 'cuda',
           'META_ARCHITECTURE': 'GeneralizedRCNN',
           'CLS_AGNOSTIC_BBOX_REG': False,
           'WEIGHT': 'https://download.pytorch.org/models/vgg16-397923af.pth',
           'PRETRAINED_DETECTOR_CKPT': 'https://download.pytorch.org/models/vgg16-397923af.pth',
           'PRETRAINED_MODEL_CKPT': '',
           'BACKBONE': {'CONV_BODY': 'VGG-16',
                        'FREEZE_CONV_BODY_AT': 2},
           'FPN': CfgNode({'USE_GN': False, 'USE_RELU': False}),
           'GROUP_NORM': {'DIM_PER_GP': -1,
                          'NUM_GROUPS': 32,
                          'EPSILON': 1e-05},
           'RPN': {'USE_FPN': False,
                   'RPN_MID_CHANNEL': 512,
                   'ANCHOR_SIZES': (32, 64, 128, 256, 512),
   

In [12]:
model = build_detection_model(cfg)
eval_modules = (model.rpn, model.backbone, model.roi_heads.box,)
fix_eval_modules(eval_modules)

rel_classes=['__background__', 'above', 'across', 'against', 'along', 'and', 'at', 'attached to', 'behind', 'belonging to', 'between', 'carrying', 'covered in', 'covering', 'eating', 'flying in', 'for', 'from', 'growing on', 'hanging from', 'has', 'holding', 'in', 'in front of', 'laying on', 'looking at', 'lying on', 'made of', 'mounted on', 'near', 'of', 'on', 'on back of', 'over', 'painted on', 'parked on', 'part of', 'playing', 'riding', 'says', 'sitting on', 'standing on', 'to', 'under', 'using', 'walking in', 'walking on', 'watching', 'wearing', 'wears', 'with']


In [14]:
from torchinfo import summary

In [15]:
# input_size = (batch_size, 1, 28, 28)

In [16]:
summary(model)

Layer (type:depth-idx)                             Param #
GeneralizedRCNN                                    --
├─Sequential: 1-1                                  --
│    └─VGG16: 2-1                                  --
│    │    └─Sequential: 3-1                        (14,714,688)
├─RPNModule: 1-2                                   --
│    └─AnchorGenerator: 2-2                        --
│    │    └─BufferList: 3-2                        --
│    └─RPNHead: 2-3                                --
│    │    └─Conv2d: 3-3                            (2,359,808)
│    │    └─Conv2d: 3-4                            (10,260)
│    │    └─Conv2d: 3-5                            (41,040)
│    └─RPNPostProcessor: 2-4                       --
│    └─RPNPostProcessor: 2-5                       --
├─CombinedROIHeads: 1-3                            --
│    └─ROIBoxHead: 2-6                             --
│    │    └─FPN2MLPFeatureExtractor: 3-6           (119,545,856)
│    │    └─FPNPredictor: 3-7      

In [21]:
model

GeneralizedRCNN(
  (backbone): Sequential(
    (body): VGG16(
      (conv_body): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace=True)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace=True)
        (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace=True)
        (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace=True)
        (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (13): ReLU(inplace=True)
        (14): Conv2d(256, 256, kernel_size=(3, 3), stri

In [20]:
model.backbone

Sequential(
  (body): VGG16(
    (conv_body): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (

In [17]:
summary(model.rpn)

Layer (type:depth-idx)                   Param #
RPNModule                                --
├─AnchorGenerator: 1-1                   --
│    └─BufferList: 2-1                   --
├─RPNHead: 1-2                           --
│    └─Conv2d: 2-2                       (2,359,808)
│    └─Conv2d: 2-3                       (10,260)
│    └─Conv2d: 2-4                       (41,040)
├─RPNPostProcessor: 1-3                  --
├─RPNPostProcessor: 1-4                  --
Total params: 2,411,108
Trainable params: 0
Non-trainable params: 2,411,108

In [18]:
summary(model.backbone)

Layer (type:depth-idx)                   Param #
Sequential                               --
├─VGG16: 1-1                             --
│    └─Sequential: 2-1                   --
│    │    └─Conv2d: 3-1                  (1,792)
│    │    └─ReLU: 3-2                    --
│    │    └─Conv2d: 3-3                  (36,928)
│    │    └─ReLU: 3-4                    --
│    │    └─MaxPool2d: 3-5               --
│    │    └─Conv2d: 3-6                  (73,856)
│    │    └─ReLU: 3-7                    --
│    │    └─Conv2d: 3-8                  (147,584)
│    │    └─ReLU: 3-9                    --
│    │    └─MaxPool2d: 3-10              --
│    │    └─Conv2d: 3-11                 (295,168)
│    │    └─ReLU: 3-12                   --
│    │    └─Conv2d: 3-13                 (590,080)
│    │    └─ReLU: 3-14                   --
│    │    └─Conv2d: 3-15                 (590,080)
│    │    └─ReLU: 3-16                   --
│    │    └─MaxPool2d: 3-17              --
│    │    └─Conv2d: 3-18  

In [19]:
summary(model.roi_heads.box)

Layer (type:depth-idx)                   Param #
ROIBoxHead                               --
├─FPN2MLPFeatureExtractor: 1-1           --
│    └─Pooler: 2-1                       --
│    │    └─ModuleList: 3-1              --
│    └─Linear: 2-2                       (102,764,544)
│    └─Linear: 2-3                       (16,781,312)
├─FPNPredictor: 1-2                      --
│    └─Linear: 2-4                       (618,647)
│    └─Linear: 2-5                       (2,474,588)
├─PostProcessor: 1-3                     --
Total params: 122,639,091
Trainable params: 0
Non-trainable params: 122,639,091

In [ ]:
model.roi_heads.box

In [ ]:
model.roi_heads

In [ ]:
from util_misc import print_para

In [ ]:
print(print_para(model))

# 2. Output the linalg.norm of each backbone layer weight and try to transfer from GB-Net

In [22]:
model.backbone

Sequential(
  (body): VGG16(
    (conv_body): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (

In [34]:
model.backbone.body.conv_body[0].weight.norm()

tensor(10.4271)